### 1. Imports

In [6]:
import pandas as pd
import numpy as np
import os
import openai
import nltk
import random
import re
import ast
import time



from nltk.tokenize import sent_tokenize
nltk.download('punkt')


# openai.api_key = # Enter your API KEY

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/maxencedubois/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 2. Data & Parameters

In [7]:
data = pd.read_csv('your_data.csv')

size_answers = 1200
sent_call = 100 #If split sentences, recommended size = 100, if unsplit recommended size = 50
split_sentences = "No"

### 3. Defining functions

In [8]:
def remove_apostrophe(text):
    return text.replace("'", "")

def add_backslash(text):
    return text.replace("'", "\\'")

def flatten_list(nested_list):
    flat_list = []
    for item in nested_list:
        if isinstance(item, list):
            flat_list.extend(flatten_list(item))
        else:
            flat_list.append(item)
    return flat_list

In [ ]:
def getSentiment(df,sent_call , split_sentences):
    
    if split_sentences == 'Yes':

        df['value'] = df['value'].apply(sent_tokenize)
        df = df.explode('value').reset_index(drop=True)
        df['value'] = df['value'].str.split('- ')  
        
    df = df.explode('value').reset_index(drop=True)
    df = df[df['value'].str.len()>3]


    df['value'] = df['value'].apply(add_backslash)
    
    output_df = df

    dfs = []

    for value in df['variable'].unique():
        new_df = df[df['variable'] == value].copy()
        dfs.append(new_df)

    sentiment_all = []


    for df in dfs:

        q = df['variable'].iloc[0]

        sentiments = ["Positive","Neutral", "Negative"]

        sentiment_q = []

        sent = [f"{index + 1}. {sentence}" for index, sentence in enumerate(df['value'])]
        n = len(sent)
        sublists = [sent[i:i+sent_call] for i in range(0, n, sent_call)]

        for sublist in sublists:

            sentiment_list = []

            n_sub = len(sublist)

            p = f"These sentences are answers to the question {q}. Classify the sentiment in these '{n_sub}'sentences '{sublist}' using  these sentiments '{sentiments}'. Use the question as additional context to help find the sentiment. Print me the result for each sentence (just give me the number of the sentence and the corresponding sentiment, don't print the sentence again) "


            response = openai.Completion.create(
                                             model="text-davinci-003",
                                             prompt=p,
                                             max_tokens=500,
                                             temperature=0,
                                             top_p=1.0,
                                             frequency_penalty=0.0,
                                             presence_penalty=0.0
                                            )

            text = response['choices'][0]['text'].lstrip()
            lines = text.splitlines()

            for line in lines:
                if '.' in line:
                    sentiment = line.split('. ', 1)[1].strip()
                else:
                    sentiment = line.split(' ', 1)[1].strip()

                sentiment_list.append(sentiment)

            sentiment_q.append(sentiment_list)

        sentiment_all.append(sentiment_q)

    sentiment_all = flatten_list(sentiment_all)
    
    output_df['sentiment_GPT'] = sentiment_all
    
    output_df.to_excel('data_sentiment.xlsx', index=False)
    output_df.to_csv('data_sentiment.csv', index=False)
    
    
    return output_df

### 4. Results

In [10]:
sentiment_df = getSentiment(data, sent_call , split_sentences)